### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [272]:
tot_sch=len(data.school_name.unique())
tot_stu=len(data['Student ID'].unique())

budget=data[['school_name','budget']].drop_duplicates()
tot_bud=budget.budget.sum()

mat_avg=data['math_score'].mean()
read_avg=data['reading_score'].mean()

pct_mat=len(data.math_score[data.math_score>=70])*100/tot_stu

pct_read=len(data.reading_score[data.reading_score>=70])*100/tot_stu

tot_pass=data[(data.reading_score>=70) & (data.math_score>=70)].shape[0]*100/tot_stu

dist_sum=pd.DataFrame({
    'Total Schools':[tot_sch],
    'Total Students':[tot_stu],
    'Total Budget':[tot_bud],
    'Average Math Score':[mat_avg],
    'Average Reading Score':[read_avg],
    '% Passing reading':[pct_read],
    '% Passing Math':[pct_mat],
    '% Overall Passing':[tot_pass] 
})

dist_sum.style.format({'Total Students': "{:,}",
                       'Total Budget': "${:,.2f}",
                       'Average Math Score':"{:,.2f}",
                       'Average Reading Score':"{:,.2f}",
                       '% Passing reading':"{:,.2f}",
                       '% Passing Math':"{:,.2f}",
                       '% Overall Passing':"{:,.2f}" 
                      })


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing reading,% Passing Math,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,85.81,74.98,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [273]:
#Creating table 1 with totals
table1=data[['school_name','type','Student ID','budget']].groupby(['school_name','type','budget'],as_index=False).count()
table1.rename(columns={'Student ID':'tot_stu'},inplace=True)
table1['bud_alum']=table1['budget']/table1['tot_stu']

#creating with averages
table2=data[['school_name','math_score','reading_score']].groupby('school_name',as_index=False).mean()

#creating table with passing scores
bins=[0,69.999999999999,100]

data['m_pass']=pd.cut(data.math_score,bins=bins,labels=False)*100
data['r_pass']=pd.cut(data.reading_score,bins=bins,labels=False)*100
data['ov_pass']=(data['m_pass']+data['r_pass']-100)
data['ov_pass'].replace(-100,0,inplace=True)

table3=data[['school_name','m_pass','r_pass','ov_pass']].groupby('school_name',as_index=False).mean()

In [274]:
school_sum=table1.merge(table2,on='school_name').merge(table3,on='school_name')

In [290]:
school_sum.rename(columns={'school_name':'School name',
                          'type':'School Type',
                          'tot_stu':'Total Students',
                          'budget':'Total School Budget',
                          'bud_alum':'Per Student Budget',
                          'math_score':'Average Math Score',
                          'reading_score':'Average Reading Score',
                          'm_pass':'% Passing Math',
                          'r_pass':'% Passing Reading',
                          'ov_pass':'% Overall Passing'},
                          inplace=True
                         )

school_sum=school_sum.style.format({'Total Students':"{:,}",
                         'Total School Budget':"${:,.2f}",
                         'Per Student Budget':"${:,.2f}",
                         'Average Math Score':"{:,.2f}",
                         'Average Reading Score':"{:,.2f}",
                         '% Passing Math':"{:,.2f}",
                         '% Passing Reading':"{:,.2f}",
                         '% Overall Passing':"{:,.2f}"
                      })

display(school_sum)

,School name,School Type,Total School Budget,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,"$3,124,928.00","4,976",$628.00,77.05,81.03,66.68,81.93,54.64
1,Cabrera High School,Charter,"$1,081,356.00","1,858",$582.00,83.06,83.98,94.13,97.04,91.33
2,Figueroa High School,District,"$1,884,411.00","2,949",$639.00,76.71,81.16,65.99,80.74,53.20
3,Ford High School,District,"$1,763,916.00","2,739",$644.00,77.10,80.75,68.31,79.30,54.29
4,Griffin High School,Charter,"$917,500.00","1,468",$625.00,83.35,83.82,93.39,97.14,90.60
5,Hernandez High School,District,"$3,022,020.00","4,635",$652.00,77.29,80.93,66.75,80.86,53.53
6,Holden High School,Charter,"$248,087.00",427,$581.00,83.80,83.81,92.51,96.25,89.23
7,Huang High School,District,"$1,910,635.00","2,917",$655.00,76.63,81.18,65.68,81.32,53.51
8,Johnson High School,District,"$3,094,650.00","4,761",$650.00,77.07,80.97,66.06,81.22,53.54
9,Pena High School,Charter,"$585,858.00",962,$609.00,83.84,84.04,94.59,95.95,90.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [276]:
school_sum.sort_values('% Overall Passing',ascending=False,inplace=True)
best=school_sum.head(5).reset_index(drop=True)
best.style.format({'Total Students':"{:,}",
                         'Total School Budgent':"${:,.2f}",
                         'Per Student Budget':"${:,.2f}",
                         'Average Math Score':"{:,.2f}",
                         'Average Reading Score':"{:,.2f}",
                         '% Passing Math':"{:,.2f}",
                         '% Passing Reading':"{:,.2f}",
                         '% Overall Passing':"{:,.2f}"
                      })


,School name,School Type,Total School Budgent,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Cabrera High School,Charter,"$1,081,356.00","1,858",$582.00,83.06,83.98,94.13,97.04,91.33
1,Thomas High School,Charter,"$1,043,130.00","1,635",$638.00,83.42,83.85,93.27,97.31,90.95
2,Griffin High School,Charter,"$917,500.00","1,468",$625.00,83.35,83.82,93.39,97.14,90.60
3,Wilson High School,Charter,"$1,319,574.00","2,283",$578.00,83.27,83.99,93.87,96.54,90.58
4,Pena High School,Charter,"$585,858.00",962,$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [277]:
school_sum.sort_values('% Overall Passing',ascending=True,inplace=True)
worst=school_sum.head(5).reset_index(drop=True)
worst.style.format({'Total Students':"{:,}",
                         'Total School Budgent':"${:,.2f}",
                         'Per Student Budget':"${:,.2f}",
                         'Average Math Score':"{:,.2f}",
                         'Average Reading Score':"{:,.2f}",
                         '% Passing Math':"{:,.2f}",
                         '% Passing Reading':"{:,.2f}",
                         '% Overall Passing':"{:,.2f}"
                      })

,School name,School Type,Total School Budgent,Total Students,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Rodriguez High School,District,"$2,547,363.00","3,999",$637.00,76.84,80.74,66.37,80.22,52.99
1,Figueroa High School,District,"$1,884,411.00","2,949",$639.00,76.71,81.16,65.99,80.74,53.20
2,Huang High School,District,"$1,910,635.00","2,917",$655.00,76.63,81.18,65.68,81.32,53.51
3,Hernandez High School,District,"$3,022,020.00","4,635",$652.00,77.29,80.93,66.75,80.86,53.53
4,Johnson High School,District,"$3,094,650.00","4,761",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [278]:
import numpy as np

math_sc=pd.pivot_table(data=data, values='math_score', index=['school_name'],
                    columns=['grade'], aggfunc=np.mean)
math_sc.reset_index(inplace=True)
math_sc=math_sc[['school_name','9th','10th','11th','12th']]
math_sc.rename_axis('index',axis=1,inplace=True)

math_sc.rename(columns={'school_name':'School Name'},inplace=True)

math_sc.style.format({'9th':"{:,.2f}",
                      '10th':"{:,.2f}",
                      '11th':"{:,.2f}",
                      '12th':"{:,.2f}"})

index,School Name,9th,10th,11th,12th
0,Bailey High School,77.08,77.00,77.52,76.49
1,Cabrera High School,83.09,83.15,82.77,83.28
2,Figueroa High School,76.40,76.54,76.88,77.15
3,Ford High School,77.36,77.67,76.92,76.18
4,Griffin High School,82.04,84.23,83.84,83.36
5,Hernandez High School,77.44,77.34,77.14,77.19
6,Holden High School,83.79,83.43,85.00,82.86
7,Huang High School,77.03,75.91,76.45,77.23
8,Johnson High School,77.19,76.69,77.49,76.86
9,Pena High School,83.63,83.37,84.33,84.12


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [279]:
read_sc=pd.pivot_table(data=data, values='reading_score', index=['school_name'],
                    columns=['grade'], aggfunc=np.mean)
read_sc.reset_index(inplace=True)
read_sc=read_sc[['school_name','9th','10th','11th','12th']]
read_sc.rename_axis('index',axis=1,inplace=True)

read_sc.rename(columns={'school_name':'School Name'},inplace=True)

read_sc.style.format({'9th':"{:,.2f}",
                      '10th':"{:,.2f}",
                      '11th':"{:,.2f}",
                      '12th':"{:,.2f}"})

index,School Name,9th,10th,11th,12th
0,Bailey High School,81.30,80.91,80.95,80.91
1,Cabrera High School,83.68,84.25,83.79,84.29
2,Figueroa High School,81.20,81.41,80.64,81.38
3,Ford High School,80.63,81.26,80.40,80.66
4,Griffin High School,83.37,83.71,84.29,84.01
5,Hernandez High School,80.87,80.66,81.40,80.86
6,Holden High School,83.68,83.32,83.82,84.70
7,Huang High School,81.29,81.51,81.42,80.31
8,Johnson High School,81.26,80.77,80.62,81.23
9,Pena High School,83.81,83.61,84.34,84.59


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [280]:
school_sum=table1.merge(table2,on='school_name').merge(table3,on='school_name')
bins=[0,585,630,645,680]
labels='<$585 $585-630 $630-645 $645-680'.split()
school_sum['spend']=pd.cut(school_sum['bud_alum'],bins=bins,labels=labels,right=False)
s_by_sch=school_sum[['spend','math_score','reading_score','m_pass','r_pass','ov_pass']].groupby('spend').mean()

In [281]:
s_by_sch.rename(columns={ 'math_score':'Average Math Score',
                          'reading_score':'Average Reading Score',
                          'm_pass':'% Passing Math',
                          'r_pass':'% Passing Reading',
                          'ov_pass':'% Overall Passing'},
                          inplace=True
                         )

s_by_sch.rename_axis('Spending Ranges (Per Student)',axis=0,inplace=True)

s_by_sch.style.format({'Average Math Score':"{:,.2f}",
                         'Average Reading Score':"{:,.2f}",
                         '% Passing Math':"{:,.2f}",
                         '% Passing Reading':"{:,.2f}",
                         '% Overall Passing':"{:,.2f}"
                      })

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [282]:
school_sum=table1.merge(table2,on='school_name').merge(table3,on='school_name')
bins=[0,1000,2000,5000]
labels='Small(<1000) Medium(1000-2000) Large(2000-5000)'.split()
school_sum['size']=pd.cut(school_sum['tot_stu'],bins=bins,labels=labels,right=False)
s_by_sch=school_sum[['size','math_score','reading_score','m_pass','r_pass','ov_pass']].groupby('size').mean()

In [283]:
s_by_sch.rename(columns={ 'math_score':'Average Math Score',
                          'reading_score':'Average Reading Score',
                          'm_pass':'% Passing Math',
                          'r_pass':'% Passing Reading',
                          'ov_pass':'% Overall Passing'},
                          inplace=True
                         )

s_by_sch.rename_axis('School size',axis=0,inplace=True)

s_by_sch.style.format({'Average Math Score':"{:,.2f}",
                         'Average Reading Score':"{:,.2f}",
                         '% Passing Math':"{:,.2f}",
                         '% Passing Reading':"{:,.2f}",
                         '% Overall Passing':"{:,.2f}"
                      })

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School size,,,,,
Small(<1000),83.82,83.93,93.55,96.10,89.88
Medium(1000-2000),83.37,83.86,93.60,96.79,90.62
Large(2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [284]:
school_sum=table1.merge(table2,on='school_name').merge(table3,on='school_name')
s_by_sch=school_sum[['type','math_score','reading_score','m_pass','r_pass','ov_pass']].groupby('type').mean()

In [285]:
s_by_sch.rename(columns={ 'math_score':'Average Math Score',
                          'reading_score':'Average Reading Score',
                          'm_pass':'% Passing Math',
                          'r_pass':'% Passing Reading',
                          'ov_pass':'% Overall Passing'},
                          inplace=True
                         )

s_by_sch.rename_axis('School Type',axis=0,inplace=True)

s_by_sch.style.format({'Average Math Score':"{:,.2f}",
                         'Average Reading Score':"{:,.2f}",
                         '% Passing Math':"{:,.2f}",
                         '% Passing Reading':"{:,.2f}",
                         '% Overall Passing':"{:,.2f}"
                      })

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67


### Conclusions

In the 15 schools analyzed students performed worst in math than in reading, an expected result. Also, the larger the school the worst are the scores. This can be due to a disproportionated ratio between teachers and students in these schools. 

I also found a counterintuitive finding: on average schools with the best overall performance are the schools with less spending per student. We need to be cautious about the implications, spending less money doesn't mean better scores. 

A time-series analysis would be helpful. Suppose is 2018 and the Government is experimenting with the school budgets to see the impact in the math and reading scores. The government gave randomly more money to a certain number of schools than others. In 2019, the government got the results and saw a huge impact: X%  scores' increase for each Y% in budget's increase. The experiment worked, so in 2020 the Government decides to give more money to those schools who didn't receive it in the first round, and also they decrease the budget in the schools who did receive the money -even the government has budget's constrains. 

In this hyphotetical scenario, we'll see schools with shorter budget performing better and that doesn't mean money didn't help the students to improve their performance. 